In [10]:
import xgboost as xgb
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# ================== Load and Preprocess Data ==================
df = pd.read_excel(r"C:\Users\Tanvi Parihar\OneDrive\Documents\hydrogen embrittlement dataset .xlsx")
# Drop columns with all missing values
df = df.dropna(axis=1, how='all')

# Fill any remaining NaNs
df = df.fillna(0)

# Define features and target
X = df.drop(columns=['ys'])
y = df['ys']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# ================== XGBoost with Hyperparameter Tuning ==================
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

param_grid = {
    'n_estimators': [200,400],
    'max_depth': [3, 5],
    'learning_rate': [0.03, 0.05],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'reg_alpha': [0.0, 0.1, 0.5, 1],   # L1 regularization (lasso)
    'reg_lambda': [1,5]    # L2 regularization (ridge)
}
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

# ================== Predictions ==================
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# ================== Evaluation ==================
print("Best Parameters:", grid_search.best_params_)
print("Train MSE:", mean_squared_error(y_train, y_train_pred))
print("Train R²:", r2_score(y_train, y_train_pred))
print("Test MSE:", mean_squared_error(y_test, y_test_pred))
print("Test R²:", r2_score(y_test, y_test_pred))


Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 200, 'reg_alpha': 0.0, 'reg_lambda': 5, 'subsample': 0.8}
Train MSE: 6742.067585708921
Train R²: 0.9324359944521288
Test MSE: 41740.43965775887
Test R²: 0.6017122671762289


In [15]:
import xgboost as xgb
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# ================== Load and Preprocess Data ==================
df = pd.read_excel(r"C:\Users\Tanvi Parihar\OneDrive\Documents\hydrogen embrittlement dataset .xlsx")
# Drop columns with all missing values
df = df.dropna(axis=1, how='all')

# Fill any remaining NaNs
df = df.fillna(0)

# Define features and target
X = df.drop(columns=['ys'])
y = df['ys']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# ================== XGBoost with Hyperparameter Tuning ==================
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

param_grid = {
    'n_estimators': [200,400],
    'max_depth': [3, 5],
    'learning_rate': [0.03, 0.05],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'reg_alpha': [0.0, 0.1, 0.5, 1],   # L1 regularization (lasso)
    'reg_lambda': [1,5]    # L2 regularization (ridge)
}
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

# ================== Predictions ==================
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# ================== Evaluation ==================
print("Best Parameters:", grid_search.best_params_)
print("Train MSE:", mean_squared_error(y_train, y_train_pred))
print("Train R²:", r2_score(y_train, y_train_pred))
print("Test MSE:", mean_squared_error(y_test, y_test_pred))
print("Test R²:", r2_score(y_test, y_test_pred))

# ================== Reverse Design ==================
print("Starting reverse design...")

# Target properties
target_ys = 850  # MPa
tolerance = 25   # ± range

print(f"Target ys: {target_ys} ± {tolerance} MPa")
print(f"Training data ys range: {y_train.min():.2f} - {y_train.max():.2f} MPa")

# Generate random samples
n_samples = 50000  # Increased for better coverage
print(f"Generating {n_samples} random samples...")

# Expected features based on trained model
expected_features = best_model.feature_names_in_
samples = pd.DataFrame()

# Generate samples using normal distribution around feature means
for feature in expected_features:
    col_data = X[feature].dropna()
    min_val, max_val = col_data.min(), col_data.max()
    mean_val, std_val = col_data.mean(), col_data.std()
    
    if np.issubdtype(col_data.dtype, np.number):
        # Use normal distribution for better sampling
        samples[feature] = np.random.normal(mean_val, std_val/2, n_samples)
        # Clip to data bounds
        samples[feature] = np.clip(samples[feature], min_val, max_val)
    else:
        # For categorical features
        samples[feature] = np.random.choice(col_data.unique(), n_samples)

# Predict using best_model
predicted_ys = best_model.predict(samples)
print(f"Predicted ys range: {predicted_ys.min():.2f} - {predicted_ys.max():.2f} MPa")

# Filter compositions matching the target
mask = (predicted_ys >= target_ys - tolerance) & (predicted_ys <= target_ys + tolerance)
matches = samples[mask].copy()

if len(matches) > 0:
    matches['predicted_ys'] = predicted_ys[mask]
    matches = matches.sort_values('predicted_ys')
    
    print(f"\n Found {len(matches)} candidate compositions for target ys ≈ {target_ys} MPa")
    print()
    
    # Display results in table format like your example
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    
    # Show only a reasonable number of results (top 10) and reset index
    display_matches = matches.head(10) if len(matches) > 10 else matches
    display_matches = display_matches.reset_index(drop=True)  # Remove original index numbers
    print(display_matches.round(6))
    
    # Find and highlight the BEST composition (closest to target)
    best_idx = (matches['predicted_ys'] - target_ys).abs().idxmin()
    best_composition = matches.loc[best_idx]
    
    print(f"\n BEST COMPOSITION (closest to target {target_ys} MPa):")
    print("=" * 60)
    print(f"Predicted ys: {best_composition['predicted_ys']:.2f} MPa")
    print(f"Deviation from target: {abs(best_composition['predicted_ys'] - target_ys):.2f} MPa")
    print("\nOptimal feature values:")
    for feature in expected_features:
        print(f"  {feature}: {best_composition[feature]:.6f}")
    print("=" * 60)
    
else:
    print(f"\n No compositions found for target {target_ys} ± {tolerance} MPa")
    print("Try:")
    print(f"1. Increase tolerance (current: ±{tolerance})")
    print(f"2. Adjust target value (current: {target_ys})")
    print(f"3. Increase samples (current: {n_samples})")
    
    # Show closest match
    closest_idx = np.argmin(np.abs(predicted_ys - target_ys))
    print(f"\nClosest prediction: {predicted_ys[closest_idx]:.2f} MPa")

Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 200, 'reg_alpha': 0.0, 'reg_lambda': 5, 'subsample': 0.8}
Train MSE: 6742.067585708921
Train R²: 0.9324359944521288
Test MSE: 41740.43965775887
Test R²: 0.6017122671762289
Starting reverse design...
Target ys: 850 ± 25 MPa
Training data ys range: 0.00 - 1550.00 MPa
Generating 50000 random samples...
Predicted ys range: 216.00 - 1143.75 MPa

✅ Found 1635 candidate compositions for target ys ≈ 850 MPa

          Fe        Ni         C        Mn        Cr        Mo        Si  \
0  96.499622  1.305318  0.335696  0.404061  6.296015  0.550111  0.458155   
1  92.259716  4.557561  0.261543  0.000000  3.007011  0.218132  0.280243   
2  92.024102  4.908673  0.304609  0.000000  1.484753  0.321953  0.081117   
3  97.882551  2.266023  0.268168  0.472544  0.000000  0.178480  0.150878   
4  94.354308  2.162377  0.472538  0.000000  9.919571  0.341934  0.187689   
5  92.195093  2.206426  0.158615  1.089791

In [19]:
# %%
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from skopt import Optimizer
from skopt.space import Real
import warnings

warnings.filterwarnings('ignore')

# ========== Load Dataset ==========
df = pd.read_excel(r"C:\Users\Tanvi Parihar\OneDrive\Documents\hydrogen embrittlement dataset .xlsx")

# Replace inf/-inf and drop NaNs
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

# ========== Features and Target ==========
opt_params = ['Fe', 'Ni', 'C', 'Mn', 'Cr', 'Mo', 'Si',
              'atomic concentration', 'electron affinity', 'ionic radii',
              'work function', 'first ionization', 'valence electron', 'enthalpy of vacancies']

target = 'ys'  # Single target now

X = df[opt_params]
y = df[target]

# ========== Train/Test Split ==========
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ========== Standardization ==========
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ========== Train Model ==========
model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train_scaled, y_train)

# ========== Evaluate ==========
print("MSE (ys):", mean_squared_error(y_test, model.predict(X_test_scaled)))

# ========== Define Optimization Space ==========
param_bounds = {
    'Fe': (60, 100), 'Ni': (0, 20), 'C': (0, 3), 'Mn': (0, 30), 'Cr': (0, 20),
    'Mo': (0, 5), 'Si': (0, 2),
    'atomic concentration': (8e22, 2e24),
    'electron affinity': (50, 140),
    'ionic radii': (0.68, 0.81),
    'work function': (4.3, 5),
    'first ionization': (1200, 6000),
    'valence electron': (4.4, 5),
    'enthalpy of vacancies': (130, 180)
}

dimensions = [Real(*param_bounds[k]) for k in opt_params]

# ========== Define Objective Function ==========
def objective(params):
    df_params = pd.DataFrame([params], columns=opt_params)
    df_scaled = scaler.transform(df_params)
    pred = model.predict(df_scaled)[0]
    return -pred  # We want to maximize ys

# ========== Bayesian Optimization Loop ==========
best_results = []

for run_id in range(3):  # Try 3 different seeds
    optimizer = Optimizer(dimensions=dimensions, base_estimator='GP', n_initial_points=10, random_state=42 + run_id)

    for step in range(50):  
        next_point = optimizer.ask()
        loss = objective(next_point)
        optimizer.tell(next_point, loss)

    best = optimizer.get_result().x
    best_dict = dict(zip(opt_params, best))
    best_scaled = scaler.transform(pd.DataFrame([best_dict]))
    best_pred = model.predict(best_scaled)[0]

    best_results.append((best_dict, best_pred))

# ========== Display Results ==========
for i, (params, pred) in enumerate(best_results, 1):
    print(f"\nRun {i}:")
    print("Best Parameters:")
    for k, v in params.items():
        print(f"  {k}: {v:.4f}")
    print(f"Predicted ys: {pred:.2f}")


MSE (ys): 13286.765104062646

Run 1:
Best Parameters:
  Fe: 88.2230
  Ni: 12.2102
  C: 1.8896
  Mn: 0.0000
  Cr: 14.6287
  Mo: 3.5862
  Si: 1.4699
  atomic concentration: 265604703058984150499328.0000
  electron affinity: 140.0000
  ionic radii: 0.8100
  work function: 4.6123
  first ionization: 4114.2606
  valence electron: 4.5300
  enthalpy of vacancies: 180.0000
Predicted ys: 1128.60

Run 2:
Best Parameters:
  Fe: 84.4675
  Ni: 8.1751
  C: 3.0000
  Mn: 0.0602
  Cr: 10.6256
  Mo: 1.6324
  Si: 1.0120
  atomic concentration: 299367706326616380538880.0000
  electron affinity: 140.0000
  ionic radii: 0.8097
  work function: 5.0000
  first ionization: 4859.4761
  valence electron: 4.8866
  enthalpy of vacancies: 180.0000
Predicted ys: 1122.05

Run 3:
Best Parameters:
  Fe: 96.5278
  Ni: 17.2029
  C: 1.8815
  Mn: 2.6884
  Cr: 1.6244
  Mo: 2.2796
  Si: 1.1418
  atomic concentration: 319706509722261265580032.0000
  electron affinity: 132.4263
  ionic radii: 0.8099
  work function: 4.3902
  f